<h1 style = 'color: orange'> Used Car Sales Company - Rusty Bargain - S12  </h1>

O serviço de vendas de carros usados Rusty Bargain está desenvolvendo um aplicativo para atrair novos clientes. Nesse aplicativo, você pode descobrir rapidamente o valor de mercado do seu carro. Você tem acesso a dados históricos: especificações técnicas, versões de acabamento e preços. Você precisa construir o modelo para determinar o valor.

Diretrizes:

-   qualidade da predição;
-   velocidade da predição;
-   tempo necessário para o treinamento.

<h2 style = 'color: orange'> Initialization  </h2>

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings('ignore')

import math

import seaborn as sns

import sklearn.metrics
import sklearn.neighbors
import sklearn.preprocessing
import sklearn.linear_model

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder


from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score

from sklearn.neighbors import KNeighborsClassifier

from numpy.linalg import LinAlgError

from IPython.display import display

from catboost import CatBoostRegressor

import time


<h2 style = 'color: orange'> Reading Files:</h2>

In [2]:
df = pd.read_csv('car_data.csv')

In [3]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
df.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


A primeira vista, o DataFrame possui dados faltantes e aparentemente problemas com informações (por exemplo a coluna 'registration_year').

<h2 style = 'color: orange'> Data preprocessing:</h2>

In [6]:
# Columns renaming:

new_columns = []

for names in df.columns:
    names = names.lower()
    names = names.strip()
    new_columns.append(names)

df.columns = new_columns

In [7]:
df.columns

Index(['datecrawled', 'price', 'vehicletype', 'registrationyear', 'gearbox',
       'power', 'model', 'mileage', 'registrationmonth', 'fueltype', 'brand',
       'notrepaired', 'datecreated', 'numberofpictures', 'postalcode',
       'lastseen'],
      dtype='object')

In [8]:
new_columns_snake = {
    'datecrawled' : 'date_crawled',
    'vehicletype' : 'vehicle_type',
    'registrationyear' : "registration_year",
    'registrationmonth' : 'registration_month',
    'fueltype' : 'fuelt_ype',
    'notrepaired' : 'not_repaired',
    'datecreated' : 'date_created',
    'numberofpictures' : 'number_of_pictures',
    'postalcode' : 'postal_code',
    'lastseen' : 'last_seen'
}

df.rename(columns = new_columns_snake, inplace= True)

In [9]:
df.columns

Index(['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
       'power', 'model', 'mileage', 'registration_month', 'fuelt_ype', 'brand',
       'not_repaired', 'date_created', 'number_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In [10]:
# Handling missing values:

df.isna().mean()

date_crawled          0.000000
price                 0.000000
vehicle_type          0.105794
registration_year     0.000000
gearbox               0.055967
power                 0.000000
model                 0.055606
mileage               0.000000
registration_month    0.000000
fuelt_ype             0.092827
brand                 0.000000
not_repaired          0.200791
date_created          0.000000
number_of_pictures    0.000000
postal_code           0.000000
last_seen             0.000000
dtype: float64

In [11]:
print(df.isnull().any(axis=1).sum())

108555


In [12]:
# Analyzing the 'vehicle_type' data:

df_vehicle_type_null = df[df['vehicle_type'].isnull()]

In [13]:
df_vehicle_type_null.head()

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuelt_ype,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
16,01/04/2016 12:46,300,NaN,2016,NaN,60,polo,150000,0,petrol,volkswagen,NaN,01/04/2016 00:00,0,38871,01/04/2016 12:46
22,23/03/2016 14:52,2900,NaN,2018,manual,90,meriva,150000,5,petrol,opel,no,23/03/2016 00:00,0,49716,31/03/2016 01:16
26,10/03/2016 19:38,5555,NaN,2017,manual,125,c4,125000,4,NaN,citroen,no,10/03/2016 00:00,0,31139,16/03/2016 09:16
31,29/03/2016 16:57,899,NaN,2016,manual,60,clio,150000,6,petrol,renault,NaN,29/03/2016 00:00,0,37075,29/03/2016 17:43


In [14]:
# Filling the 'vehicle_type' based on the 'model':

model_to_vehicle_type = df.dropna(subset=['vehicle_type', 'model']).set_index('model')['vehicle_type'].to_dict()

In [15]:
df['vehicle_type'] = df.apply(lambda row: model_to_vehicle_type.get(row['model']) if pd.isnull(row['vehicle_type']) else row['vehicle_type'], axis=1)

In [16]:
print(df.isnull().any(axis=1).sum())

100463


Ao preencher os dados da coluna 'vehicle_type' com base na coluna 'model' (quando possivel), inseri informação em aprox 8000 linhas.

In [17]:
# Analyzing the 'not_repaired' data:

df_not_repaired_null = df[df['not_repaired'].isnull()]

In [18]:
df_not_repaired_null

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuelt_ype,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,24/03/2016 11:52,480,wagon,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
8,04/04/2016 23:42,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,04/04/2016 00:00,0,94505,04/04/2016 23:42
9,17/03/2016 10:53,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,17/03/2016 00:00,0,27472,31/03/2016 17:17
12,15/03/2016 22:49,999,wagon,1995,manual,115,passat,150000,11,petrol,volkswagen,NaN,15/03/2016 00:00,0,37269,01/04/2016 13:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354349,16/03/2016 17:06,5999,wagon,2005,manual,140,a4,150000,4,gasoline,audi,NaN,16/03/2016 00:00,0,56472,18/03/2016 11:30
354356,04/04/2016 11:45,999,convertible,2000,manual,95,megane,150000,4,petrol,renault,NaN,04/04/2016 00:00,0,88477,06/04/2016 12:44
354357,09/03/2016 11:36,1690,wagon,2004,manual,55,fabia,150000,4,petrol,skoda,NaN,09/03/2016 00:00,0,18246,04/04/2016 08:15
354365,14/03/2016 17:48,2200,None,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,14/03/2016 00:00,0,39576,06/04/2016 00:46


In [19]:
df['not_repaired'].value_counts(dropna=False)

not_repaired
no     247161
NaN     71154
yes     36054
Name: count, dtype: int64

Infelizmente não encontrei nenhuma referência para preenchimento dos dados faltantes da coluna 'not_repaired'. 

Essa coluna tem influência direta sobre o estado do veículo e sem o detalhamento dessa condição por parte do enunciado, não posso utilizar esses dados no modelo.

In [20]:
# Analyzing the 'number_of_pictures' data:

df['number_of_pictures'].describe()

count    354369.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: number_of_pictures, dtype: float64

Ao analizar os dados da coluna 'not_repaired' a coluna 'number_of_pictures' chamou atenção. O analiza-la percebi que todos os seus dados são ZERO. A quantidade de fotos também apenas seria relevante caso o modelo que será usado fosse capaz de analiza-las para precificação.

In [21]:
# Analyzing possible 'registration_year' errors:

df['year_crawled'] = pd.to_datetime(df['date_crawled']).dt.year
df['year_profile'] = pd.to_datetime(df['date_crawled']).dt.year

In [22]:
df[['year_crawled', 'year_profile']].value_counts()

year_crawled  year_profile
2016          2016            354369
Name: count, dtype: int64

In [23]:
print(df.query('registration_year > 2016 or registration_year < 1886').value_counts())

date_crawled      price  vehicle_type  registration_year  gearbox  power  model     mileage  registration_month  fuelt_ype  brand          not_repaired  date_created      number_of_pictures  postal_code  last_seen         year_crawled  year_profile
01/04/2016 00:46  18000  wagon         2017               auto     231    3er       150000   3                   gasoline   bmw            no            31/03/2016 00:00  0                   24536        04/04/2016 23:45  2016          2016            1
21/03/2016 20:58  4999   bus           2017               manual   0      scenic    90000    12                  petrol     renault        no            21/03/2016 00:00  0                   47239        25/03/2016 19:18  2016          2016            1
21/03/2016 19:53  900    small         2018               manual   60     ka        90000    0                   petrol     ford           no            21/03/2016 00:00  0                   20457        06/04/2016 22:46  2016          2016   

In [24]:
df = df.query('registration_year <= 2016 and registration_year > 1886')

In [25]:
df['registration_year'].describe()

count    339773.000000
mean       2002.482222
std           7.091181
min        1910.000000
25%        1999.000000
50%        2003.000000
75%        2007.000000
max        2016.000000
Name: registration_year, dtype: float64

No início da análise percebi que existiam dados de ano de registro que não faziam sentido (ex: 9999 e 1000). Ao mesmo tempo percebo que como o ano em que os dados foram baixados e os perfis criados foi o ano de 2016, não seria possível ter registro feitos no futuro (maiores que 2016). Usei também o ano de criação do carro para eliminar registos inferiores. Foram exluídas do DataFrame aprox. 4000 linhas.

O ano é uma das principais características para avaliação do veículo, por isso considero ser valida a retirada de linhas que podem apresentar qualquer erro.

In [26]:
df = df.dropna().reset_index(drop=True)

Após o preenchimento dos dados faltantes possíveis, precisei retirar do DataFrame as linhas que continham dados nulos (todas colunas relevantes para a precificação do veículo).

In [27]:
# Handling with duplicate data:

df.duplicated().sum()

249

In [28]:
df = df.drop_duplicates()

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 249564 entries, 0 to 249812
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        249564 non-null  object
 1   price               249564 non-null  int64 
 2   vehicle_type        249564 non-null  object
 3   registration_year   249564 non-null  int64 
 4   gearbox             249564 non-null  object
 5   power               249564 non-null  int64 
 6   model               249564 non-null  object
 7   mileage             249564 non-null  int64 
 8   registration_month  249564 non-null  int64 
 9   fuelt_ype           249564 non-null  object
 10  brand               249564 non-null  object
 11  not_repaired        249564 non-null  object
 12  date_created        249564 non-null  object
 13  number_of_pictures  249564 non-null  int64 
 14  postal_code         249564 non-null  int64 
 15  last_seen           249564 non-null  object
 16  year_cr

In [30]:
df_ml = df[['price',
            'vehicle_type', 
            'registration_year', 
            'gearbox', 
            'power', 
            'model', 
            'mileage', 
            'registration_month', 
            'fuelt_ype', 
            'brand', 
            'not_repaired', 
            'postal_code']]

In [31]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
Index: 249564 entries, 0 to 249812
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               249564 non-null  int64 
 1   vehicle_type        249564 non-null  object
 2   registration_year   249564 non-null  int64 
 3   gearbox             249564 non-null  object
 4   power               249564 non-null  int64 
 5   model               249564 non-null  object
 6   mileage             249564 non-null  int64 
 7   registration_month  249564 non-null  int64 
 8   fuelt_ype           249564 non-null  object
 9   brand               249564 non-null  object
 10  not_repaired        249564 non-null  object
 11  postal_code         249564 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 24.8+ MB


Entendo que um grande percentual de informação foi descartado, porém acredito que ainda há informação suficiente para o modelo nas 249.564 linhas do DataFrame.

<h2 style = 'color: orange'> ML:</h2>

In [32]:
df_ml.head(3)

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuelt_ype,brand,not_repaired,postal_code
0,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,91074
1,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,60437
2,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,33775


In [33]:
# Using pandas.get_dummies for one-hot encoding:

df_ml_encoded = pd.get_dummies(df_ml, columns=['vehicle_type', 'gearbox', 'model', 'fuelt_ype', 'brand', 'not_repaired'])

In [34]:
df_ml_encoded.shape

(249564, 313)

In [35]:
# Using label_encoding:

def apply_label_encoding(df, columns):
    
    le = LabelEncoder()
    df_encoded = df.copy()
    
    for col in columns:
        df_encoded[col] = le.fit_transform(df_encoded[col])
    
    return df_encoded

In [36]:
categorical_features_ordinal = ['registration_month', 'postal_code']

df_ml_encoded = apply_label_encoding(df_ml_encoded, categorical_features_ordinal)

In [37]:
df_ml_encoded.head()

,price,registration_year,power,mileage,registration_month,postal_code,vehicle_type_bus,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,vehicle_type_suv,vehicle_type_wagon,gearbox_auto,gearbox_manual,model_100,model_145,model_147,model_156,model_159,model_1_reihe,model_1er,model_200,model_2_reihe,model_300c,model_3_reihe,model_3er,model_4_reihe,model_500,model_5_reihe,model_5er,model_601,model_6_reihe,model_6er,model_7er,model_80,model_850,model_90,model_900,model_9000,model_911,model_a1,model_a2,model_a3,model_a4,model_a5,model_a6,model_a8,model_a_klasse,model_accord,model_agila,model_alhambra,model_almera,model_altea,model_amarok,model_antara,model_arosa,model_astra,model_auris,model_avensis,model_aveo,model_aygo,model_b_klasse,model_b_max,model_beetle,model_berlingo,model_bora,model_boxster,model_bravo,model_c1,model_c2,model_c3,model_c4,model_c5,model_c_klasse,model_c_max,model_c_reihe,model_caddy,model_calibra,model_captiva,model_carisma,model_carnival,model_cayenne,model_cc,model_ceed,model_charade,model_cherokee,model_citigo,model_civic,model_cl,model_clio,model_clk,model_clubman,model_colt,model_combo,model_cooper,model_cordoba,model_corolla,model_corsa,model_cr_reihe,model_croma,model_crossfire,model_cuore,model_cx_reihe,model_defender,model_delta,model_discovery,model_doblo,model_ducato,model_duster,model_e_klasse,model_elefantino,model_eos,model_escort,model_espace,model_exeo,model_fabia,model_fiesta,model_focus,model_forester,model_forfour,model_fortwo,model_fox,model_freelander,model_fusion,model_g_klasse,model_galant,model_galaxy,model_getz,model_gl,model_glk,model_golf,model_grand,model_i3,model_i_reihe,model_ibiza,model_impreza,model_insignia,model_jazz,model_jetta,model_jimny,model_juke,model_justy,model_ka,model_kadett,model_kaefer,model_kalina,model_kalos,model_kangoo,model_kappa,model_kuga,model_laguna,model_lancer,model_lanos,model_legacy,model_leon,model_lodgy,model_logan,model_lupo,model_lybra,model_m_klasse,model_m_reihe,model_materia,model_matiz,model_megane,model_meriva,model_micra,model_mii,model_modus,model_mondeo,model_move,model_musa,model_mustang,model_mx_reihe,model_navara,model_niva,model_note,model_nubira,model_octavia,model_omega,model_one,model_other,model_outlander,model_pajero,model_panda,model_passat,model_phaeton,model_picanto,model_polo,model_primera,model_ptcruiser,model_punto,model_q3,model_q5,model_q7,model_qashqai,model_r19,model_range_rover,model_range_rover_evoque,model_range_rover_sport,model_rangerover,model_rav,model_rio,model_roadster,model_roomster,model_rx_reihe,model_s60,model_s_klasse,model_s_max,model_s_type,model_samara,model_sandero,model_santa,model_scenic,model_scirocco,model_seicento,model_serie_2,model_serie_3,model_sharan,model_signum,model_sirion,model_sl,model_slk,model_sorento,model_spark,model_spider,model_sportage,model_sprinter,model_stilo,model_superb,model_swift,model_terios,model_tigra,model_tiguan,model_toledo,model_touareg,model_touran,model_transit,model_transporter,model_tt,model_tucson,model_twingo,model_up,model_v40,model_v50,model_v60,model_v70,model_v_klasse,model_vectra,model_verso,model_viano,model_vito,model_vivaro,model_voyager,model_wrangler,model_x_reihe,model_x_trail,model_x_type,model_xc_reihe,model_yaris,model_yeti,model_ypsilon,model_z_reihe,model_zafira,fuelt_ype_cng,fuelt_ype_electric,fuelt_ype_gasoline,fuelt_ype_hybrid,fuelt_ype_lpg,fuelt_ype_other,fuelt_ype_petrol,brand_alfa_romeo,brand_audi,brand_bmw,brand_chevrolet,brand_chrysler,brand_citroen,brand_dacia,brand_daewoo,brand_daihatsu,brand_fiat,brand_ford,brand_honda,brand_hyundai,brand_jaguar,brand_jeep,brand_kia,brand_lada,brand_lancia,brand_land_rover,brand_mazda,brand_mercedes_benz,brand_mini,brand_mitsubishi,brand_nissan,brand_opel,brand_peugeot,brand_porsche,brand_renault,brand_rover,brand_saab,brand_seat,brand_skoda,brand_smart,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,not_repaired_no,n

In [38]:
df_ml_encoded.shape

(249564, 313)

Nesse momento tento preservar a utilização de OHE para os dados categoricos não ordinais. Percebo que há um aumento significativo no conjunto de dados (colunas) e que isso pode significar dificuldades, principalmente para o treinamento dos modelos baseados em árvore.

Mais na frente farei uma comparação com a utilização apenas de Label Encoder nas colunas categóricas.

In [39]:
# Splitting the data into training (70%), validation (15%) and testing (15%) sets:

df_train, df_valid_test = train_test_split(df_ml_encoded, test_size=0.3, random_state=54321)

df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=54321)

In [40]:
# Declaring variables for 'features' and 'target':

train_features = df_train.drop('price', axis=1)
train_target = df_train['price']

valid_features = df_valid.drop('price', axis=1)
valid_target = df_valid['price']

test_features = df_test.drop('price', axis=1)
test_target = df_test['price']

Os modelos serão treinados o validados com o mesmo conjunto de dados. Percebo que isso pode causar interferência nos resultados, porém o conjunto de teste será usado apenas com o modelo escolhido e esse tem o função de validar o desempenho do modelo.

<h4 style = 'color: orange'>Random Forest Regressor with Grid Search CV:</h4>

In [41]:
start_time = time.time()

param_grid = {
    'n_estimators': [20, 40, 60],  
    'max_depth': [10, 20, 30],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']  # 'auto' excluded to minimize overfitting
}

model = RandomForestRegressor(random_state=54321)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(train_features, train_target)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_model = RandomForestRegressor(**best_params, random_state=54321)
best_model.fit(train_features, train_target)

predictions_valid = best_model.predict(valid_features)

mse_valid = mean_squared_error(valid_target, predictions_valid)
print("Root Mean Squared Error:", np.sqrt(mse_valid))

end_time = time.time()
elapsed_time = (end_time - start_time) / 60

print(f"Runtime: {elapsed_time:.2f} minutes")

Best Hyperparameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 60}
Root Mean Squared Error: 1784.7884850941398
Runtime: 60.01 minutes


<h4 style = 'color: orange'>Linear Regression with Grid Search CV:</h4>

In [42]:
start_time = time.time()

param_grid = {
    'fit_intercept': [True, False],       
}

model = LinearRegression()

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(train_features, train_target)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_model = LinearRegression(**best_params)
best_model.fit(train_features, train_target)

predictions_valid = best_model.predict(valid_features)

mse_valid = mean_squared_error(valid_target, predictions_valid)
print("Root Mean Squared Error:", np.sqrt(mse_valid))

end_time = time.time()
elapsed_time = (end_time - start_time) / 60

print(f"Runtime: {elapsed_time:.2f} minutes")

Best Hyperparameters: {'fit_intercept': False}
Root Mean Squared Error: 2793.510491252662
Runtime: 0.57 minutes


<h4 style = 'color: orange'>Gradient Boosting (LightGBM) with Grid Search CV:</h4>

In [43]:
import lightgbm as lgb

In [44]:
start_time = time.time()

param_grid = {
    'n_estimators': [20, 40, 60],                
    'learning_rate': [0.01, 0.1, 0.2],               
    'max_depth': [10, 20, 30],                        
    'num_leaves': [20, 30, 40],                        
    'subsample': [0.8, 0.9, 1.0],                    
    'min_child_samples': [10, 20],                   
    'colsample_bytree': [0.8, 1.0]                    
}

model = lgb.LGBMRegressor(random_state=12345)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(train_features, train_target)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_model = grid_search.best_estimator_

predictions_valid = best_model.predict(valid_features)

mse_valid = mean_squared_error(valid_target, predictions_valid)
print("Root Mean Squared Error:", np.sqrt(mse_valid))

end_time = time.time()
elapsed_time = (end_time - start_time) / 60

print(f"Runtime: {elapsed_time:.2f} minutes")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010427 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1202
[LightGBM] [Info] Number of data points in the train set: 174694, number of used features: 301
[LightGBM] [Info] Start training from score 5082.546624
Best Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 30, 'min_child_samples': 10, 'n_estimators': 60, 'num_leaves': 40, 'subsample': 0.8}
Root Mean Squared Error: 1722.9108311056448
Runtime: 67.52 minutes


<h4 style = 'color: orange'>CatBoost with Grid Search CV:</h4>

In [45]:
start_time = time.time()

param_grid = {
    'n_estimators': [20, 40, 60],              
    'learning_rate': [0.01, 0.1, 0.2],           
    'depth': [10, 20, 30],                       
    'subsample': [0.8, 0.9, 1.0],               
    'min_data_in_leaf': [10, 20],              
    'colsample_bylevel': [0.8, 1.0]            
}

model = CatBoostRegressor(random_state=12345, silent=True)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(train_features, train_target)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_model = grid_search.best_estimator_

predictions_valid = best_model.predict(valid_features)
mse_valid = mean_squared_error(valid_target, predictions_valid)
print("Root Mean Squared Error:", np.sqrt(mse_valid))

end_time = time.time()
elapsed_time = (end_time - start_time) / 60
print(f"Runtime: {elapsed_time:.2f} minutes")

Best Hyperparameters: {'colsample_bylevel': 1.0, 'depth': 10, 'learning_rate': 0.2, 'min_data_in_leaf': 20, 'n_estimators': 60, 'subsample': 0.8}
Root Mean Squared Error: 1762.9324784805376
Runtime: 23.95 minutes


Foram treinados 4 modelos (Random Forest Regressor, Linear Regression, Gradient Boosting (LightGBM) e CatBoost).

Resultados (RMSE / Tempo de Execução do Cod. em minutos):

-   Random Forest Regressor: 1784.78 / 60.01;
-   Linear Regression: 2793.51 / 00.57;
-   Gradient Boosting (LightGBM): 1722.91 / 67.52;
-   CatBoost:  1762.93 / 23.95.

Os 3 modelos baseados em árvore apresentam resultados superiores ao modelo de regressão linear. A prova real foi tirada.



<h3 style = 'color: orange'>Second Data Encoding (just label encoder):</h3>

Visando reduzir o tempo de execução dos modelos, resolvi fazer uma nova codificação dos dados evitando o grande numero de colunas geradas pelo OHE.

Para esses novos modelos ferei uma nova divisão dos dados.

In [46]:
df_ml.head(3)

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuelt_ype,brand,not_repaired,postal_code
0,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,91074
1,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,60437
2,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,33775


In [47]:
all_categorical_features = ['vehicle_type',  
            'gearbox', 
            'model', 
            'registration_month', 
            'fuelt_ype', 
            'brand', 
            'not_repaired', 
            'postal_code']

df_ml_encoded_2 = apply_label_encoding(df_ml, all_categorical_features)

In [48]:
df_ml_encoded_2.shape

(249564, 12)

In [49]:
# Splitting the data into training (70%), validation (15%) and testing (15%) sets:

df_train_2, df_valid_test_2 = train_test_split(df_ml_encoded_2, test_size=0.3, random_state=123)

df_valid_2, df_test_2 = train_test_split(df_ml_encoded_2, test_size=0.5, random_state=123)

In [50]:
# Declaring variables for 'features' and 'target':

train_features_2 = df_train_2.drop('price', axis=1)
train_target_2 = df_train_2['price']

valid_features_2 = df_valid_2.drop('price', axis=1)
valid_target_2 = df_valid_2['price']

test_features_2 = df_test_2.drop('price', axis=1)
test_target_2 = df_test_2['price']

<h4 style = 'color: orange'>Random Forest Regressor II with Grid Search CV:</h4>

In [51]:
start_time = time.time()

param_grid = {
    'n_estimators': [20, 40, 60],  
    'max_depth': [10, 20, 30],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']  # 'auto' excluded to minimize overfitting
}

model = RandomForestRegressor(random_state=54321)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(train_features_2, train_target_2)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_model = RandomForestRegressor(**best_params, random_state=54321)
best_model.fit(train_features_2, train_target_2)

predictions_valid = best_model.predict(valid_features_2)

mse_valid = mean_squared_error(valid_target_2, predictions_valid)
print("Root Mean Squared Error:", np.sqrt(mse_valid))

end_time = time.time()
elapsed_time = (end_time - start_time) / 60

print(f"Runtime: {elapsed_time:.2f} minutes")

Best Hyperparameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 60}
Root Mean Squared Error: 633.682566841586
Runtime: 40.79 minutes


<h4 style = 'color: orange'>Linear Regression II with Grid Search CV:</h4>

In [52]:
start_time = time.time()

param_grid = {
    'fit_intercept': [True, False],       
}

model = LinearRegression()

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(train_features_2, train_target_2)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_model = LinearRegression(**best_params)
best_model.fit(train_features_2, train_target_2)

predictions_valid_2 = best_model.predict(valid_features_2)

mse_valid = mean_squared_error(valid_target_2, predictions_valid_2)
print("Root Mean Squared Error:", np.sqrt(mse_valid))

end_time = time.time()
elapsed_time = (end_time - start_time) / 60

print(f"Runtime: {elapsed_time:.2f} minutes")

Best Hyperparameters: {'fit_intercept': True}
Root Mean Squared Error: 3399.4693665345376
Runtime: 0.02 minutes


<h4 style = 'color: orange'>Gradient Boosting II (LightGBM) with Grid Search CV:</h4>

In [53]:
start_time = time.time()

param_grid = {
    'n_estimators': [20, 40, 60],                
    'learning_rate': [0.01, 0.1, 0.2],               
    'max_depth': [10, 20, 30],                        
    'num_leaves': [20, 30, 40],                        
    'subsample': [0.8, 0.9, 1.0],                    
    'min_child_samples': [10, 20],                   
    'colsample_bytree': [0.8, 1.0]                    
}

model = lgb.LGBMRegressor(random_state=12345)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(train_features_2, train_target_2)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_model = grid_search.best_estimator_

predictions_valid_2 = best_model.predict(valid_features_2)

mse_valid = mean_squared_error(valid_target_2, predictions_valid_2)
print("Root Mean Squared Error:", np.sqrt(mse_valid))

end_time = time.time()
elapsed_time = (end_time - start_time) / 60

print(f"Runtime: {elapsed_time:.2f} minutes")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017982 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 913
[LightGBM] [Info] Number of data points in the train set: 174694, number of used features: 11
[LightGBM] [Info] Start training from score 5089.992627
Best Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 20, 'min_child_samples': 20, 'n_estimators': 60, 'num_leaves': 40, 'subsample': 0.8}
Root Mean Squared Error: 1671.817304550913
Runtime: 28.39 minutes


<h4 style = 'color: orange'>CatBoost II with Grid Search CV:</h4>

In [54]:
start_time = time.time()

param_grid = {
    'n_estimators': [20, 40, 60],              
    'learning_rate': [0.01, 0.1, 0.2],           
    'depth': [10, 20, 30],                       
    'subsample': [0.8, 0.9, 1.0],               
    'min_data_in_leaf': [10, 20],              
    'colsample_bylevel': [0.8, 1.0]            
}

model = CatBoostRegressor(random_state=12345, silent=True)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(train_features_2, train_target_2)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_model = grid_search.best_estimator_

predictions_valid_2 = best_model.predict(valid_features_2)
mse_valid = mean_squared_error(valid_target_2, predictions_valid_2)
print("Root Mean Squared Error:", np.sqrt(mse_valid))

end_time = time.time()
elapsed_time = (end_time - start_time) / 60
print(f"Runtime: {elapsed_time:.2f} minutes")

Best Hyperparameters: {'colsample_bylevel': 0.8, 'depth': 10, 'learning_rate': 0.2, 'min_data_in_leaf': 10, 'n_estimators': 60, 'subsample': 1.0}
Root Mean Squared Error: 1682.204047726458
Runtime: 16.32 minutes




Resultados comparativos (RMSE / Tempo de Execução do Cod. em minutos):

-   Random Forest Regressor: 633.68 / 40.79 (resultado anterior: 1784.78 / 60.01); 
-   Linear Regression: 3399.46 / 00.02 (resultado anterior: 2793.51 / 00.57);
-   Gradient Boosting (LightGBM): 1671.81 / 28.39 ( resultado anterior:1722.91 / 67.52);
-   CatBoost: 1682.20 / 16.32 ( resultado anterior:1762.93 / 23.95).

Todos os 4 modelos apresentaram um considerável redução no tempo de execução.

Como previsto, os modelos baseados em árvore tiveram também uma melhora no desempenho, como era previsto.

<h2 style = 'color: orange'> Chosen Model:</h2>

In [56]:
# Best Hyperparameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 60}
# Root Mean Squared Error: 633.682566841586
# Runtime: 40.79 minutes

start_time = time.time()

model_chosen = RandomForestRegressor(
    random_state=54321,
    max_depth= 30,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=60,
    max_features='sqrt'
    )

model_chosen.fit(train_features_2, train_target_2)

predictions_test = model_chosen.predict(test_features_2)

mse_valid = mean_squared_error(test_target_2, predictions_test)
print("Root Mean Squared Error:", np.sqrt(mse_valid))

end_time = time.time()
elapsed_time = (end_time - start_time) / 60

print(f"Runtime: {elapsed_time:.2f} minutes")

Root Mean Squared Error: 1304.542840679761
Runtime: 0.41 minutes


O modelo quando usado com o conjunto de teste apresenta um aumento no RMSE, porém ainda é um resultado superior aos outros modelos testados.

Essa diferença se manteve quando testada como uma nova separação dos dados.

A situação se repete mesmo depois de reiniciar o kernel e rodar apenas esse modelo, sem nenhum outro modelo ter rodado nem o conjuto de validação e nem o conjunto de teste.

Durante a escolha dos hiperparâmetros busquei sempre reduzir o risco de overfitting.

<h2 style = 'color: orange'> Conclusions:</h2>

1)
Os modelos de árvore de decisão, como a floresta aleatória e o Gradient Boosting, são capazes de lidar melhor com as variáveis categóricas codificadas por Label Encoding.

Razões pesquisadas:

-   Captura de Interações: Esses modelos podem capturar interações complexas entre variáveis. Eles não pressupõem linearidade e podem dividir o espaço de características de maneira não linear, independentemente do Label Encoding.
-   Redução de Dimensionalidade: Com OHE, o número de colunas pode aumentar significativamente, levando a um aumento na dimensionalidade. Isso pode tornar o treinamento mais demorado e, em alguns casos, pode causar overfitting. Os modelos baseados em árvores não necessariamente se beneficiam do aumento de dimensionalidade como a regressão linear.
-   Ordem Relativa: Embora Label Encoding imponha uma ordem artificial nas categorias, os modelos baseados em árvores podem lidar bem com isso, pois eles criam divisões baseadas em thresholds e não em suposições de linearidade.

2)
Os modelos de Gradient Boosting apresentam velocidade e resultado intermédios quando comaprados com os modelos de Regressão Linear e Floresta Aleatória. Porém os valores finais apresentados pelo modelo de floresta aleatória foram superiores e por isso foi o modelo escolhido.